In [1]:
from datasets import load_dataset

streaming = True


def wiki_ja_loader():
    return load_dataset("hpprc/wikipedia-20240101", split="train",
                        streaming=streaming
                        ).shuffle()



/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=iter(wiki_ja_loader())

In [48]:
noise_strings="""
存命人物
関連人物
脚注
参考文献 
"""

def clean_text(txt):
    txt=txt.replace("\n\n","\n")
    lines=txt.split("\n")
    cleaned_lines=[]
    touten_flag=False

    for line in lines[::-1]:
        if line.endswith("。"):
            touten_flag=True
        if touten_flag:
            cleaned_lines.append(line)

    cleaned_lines=cleaned_lines[::-1]


    noise_list=noise_strings.strip().split("\n")

    cleaned_lines2=[]
    stop_flag=False
    for line in cleaned_lines:
        for word in noise_list:
            if word in line:
                stop_flag=True
        if stop_flag:
            break
        cleaned_lines2.append(line)

    return "\n".join(cleaned_lines2)

In [53]:
class CleanedJapaneseWikiDataset:
    def __init__(self,streaming=True):
        self.dataset=load_dataset("hpprc/wikipedia-20240101", split="train",
                                streaming=streaming
                                ).shuffle()
        self.loader=iter(self.dataset)

    def __iter__(self):
        # イテレータは自分自身を返す
        return self

    def __next__(self):
        d=next(self.loader)
        d["text"]=clean_text(d["text"])
        return d

In [54]:
dataset=CleanedJapaneseWikiDataset()

In [55]:
loader=iter(dataset)

In [56]:
next(loader)

{'id': '1584951',
 'url': 'https://ja.wikipedia.org/wiki/%E8%97%A4%E5%8E%9F%E9%80%9A%E5%9F%BA%20%28%E5%A4%A7%E8%94%B5%E5%8D%BF%29',
 'title': '藤原通基 (大蔵卿)',
 'text': '藤原 通基（ふじわら の みちもと）は、平安時代後期の貴族。能登守・藤原基頼の次男。官位は正四位下・大蔵卿。持明院 通基とも呼ばれる。\n経歴 \n白河院政期末から鳥羽院政期前半にかけて、武蔵守・因幡守・丹波守と受領を歴任したほか、左京権大夫を務める。鳥羽院政期後半の康治2年（1143年）頃には大蔵卿に任ぜられている。久安4年（1148年）10月10日卒去。享年59。\n天治年間（1124年-1126年）において、父の基頼が創建した持仏堂である持明院を改築・拡張して、安楽光院と改名。鳥羽上皇を招いて盛大な供養を行った。これ以降「持明院」の名称はその邸宅の名として残り、またその家の家号ともなった。三男の基家がこれを継承し、その子孫は羽林家の持明院家として発展した。\n官歴 \n大治2年（1127年） 正月3日：見武蔵守。12月7日：因幡守\n大治5年（1130年） 正月6日：正五位下\n長承3年（1134年） 8月13日：見丹波守\n保延元年（1135年） 5月：見左京権大夫兼丹波守\n康治2年（1143年） 7月11日：見大蔵卿\n久安4年（1148年） 10月10日：卒去\n系譜 \n『尊卑分脈』による。'}